# Importing Values

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime as dt
import scipy as sp
import scipy.fftpack
import sklearn
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# Reading csv files

In [4]:
#reading CMGData csv file
df = pd.read_csv('CGMData.csv', low_memory=False)
df['DateTime'] = (df['Date'] + ' ' + df['Time'])
df['DateTime'] = pd.to_datetime(df['DateTime'])


#reading Insulin csv file
dfI = pd.read_csv('InsulinData.csv', low_memory=False)
dfI['DateTime'] = (dfI['Date'] + ' ' + dfI['Time'])
dfI['DateTime'] = pd.to_datetime(dfI['DateTime'])


# Extracting time series from Insulin.csv

In [5]:
#Day time range
dayTime_str = '02:00:00'
timeLimit = pd.to_timedelta(dayTime_str)

#Extarcting Meal times from Insulin.CSV
dfI_Meals_Taken = dfI.loc[dfI['BWZ Carb Input (grams)'] > 0]
mealTime_List = dfI_Meals_Taken['DateTime'].tolist()

#Extracting valid Meal Times from Insulin.CSV
def extractMealTimeInsulin(mealTime_List):
    mealTime_Valid = [mealTime_List[0]]
    for i in range(len(mealTime_List) - 1):
        diff = mealTime_List[i] - mealTime_List[i+1]
        if diff > timeLimit:
            mealTime_Valid.append(mealTime_List[i+1])
    return mealTime_Valid

#Defining valid times for both insulin files
mealTime_Valid = extractMealTimeInsulin(mealTime_List)


# Finding max and min carb values

In [6]:
carbInput = []
carbInputTime = []

def carbinputvalueExtraciton(mealTime_Valid):
    for i in range(len(mealTime_Valid)):
        dfI_carbinput = dfI.loc[dfI['DateTime'] == mealTime_Valid[i]]
        dfI_carbinput = dfI_carbinput[(dfI_carbinput['BWZ Carb Input (grams)'] > 0) & (dfI_carbinput['BWZ Carb Input (grams)'] > 2)]
        dfI_carbinput = dfI_carbinput[["BWZ Carb Input (grams)", "DateTime"]]
        carbInputval = dfI_carbinput['BWZ Carb Input (grams)'].values[0]
        carbInputTimeval = dfI_carbinput['DateTime'].values[0]
        carbInput.append(carbInputval)
        carbInputTime.append(carbInputTimeval)
    return carbInput, carbInputTime

carbInput, carbInputTime = carbinputvalueExtraciton(mealTime_Valid)

#Extracting min and max from carbInput list
carbMin = int(min(carbInput))
carbMax = int(max(carbInput))

data1 =[]
for i in range(len(carbInputTime)):
    data1.append({
    'Carb Input' : carbInput[i],
    'DateTime' : carbInputTime[i]})

dfI_carbinput = pd.DataFrame(data1)

# Extracting Valid TimeSeries

In [7]:
#Extracting time series
def extractingValidTimeSeries(mealTime_Valid):
    timeCMG_Valid = []
    timeCMG_Valid_Extra30 = []
    timeCMG_ValidStart = []
    for i in mealTime_Valid:
        df_time = df.loc[df['DateTime'] >= i]
        time_2 = df_time['DateTime'].iloc[-1]
        timeCMG_ValidStart.append(time_2)
        time_3, time_4 = time_2 - pd.Timedelta(minutes = 30), time_2 + pd.Timedelta(minutes = 120)
        timeCMG_Valid.append(time_4)
        timeCMG_Valid_Extra30.append(time_3)
    return timeCMG_Valid, timeCMG_Valid_Extra30, timeCMG_ValidStart

timeCMG_Valid, timeCMG_Valid_Extra30, timeCMG_ValidStart = extractingValidTimeSeries(mealTime_Valid)

# Extarcting Meal Matrix

In [8]:
def extractingMealMatrix(timeCMG_Valid, timeCMG_Valid_Extra30, timeCMG_ValidStart):
    meal_Matrix = []
    time_Matrix = []
    #Recorded true time
    trueMeal_Matrix = []
    trueTime_Matrix = []
    for i in range(len(timeCMG_Valid)):  
        df_valid = df.loc[df['DateTime'].between(timeCMG_Valid_Extra30[i], timeCMG_Valid[i], inclusive='both')]
        df_trueMeal = df.loc[df['DateTime'].between(timeCMG_ValidStart[i], timeCMG_Valid[i], inclusive='both')]
        df_valid1 = df_valid.loc[df_valid['Sensor Glucose (mg/dL)'] >= 0]
        df_trueMeal1 = df_trueMeal.loc[df_trueMeal['Sensor Glucose (mg/dL)'] >= 0]
    
        time_List = df_valid1['DateTime'].tolist()
        meal_List = df_valid1['Sensor Glucose (mg/dL)'].tolist()  
        trueTime_List = df_trueMeal1['DateTime'].tolist()
        trueMeal_List = df_trueMeal1['Sensor Glucose (mg/dL)'].tolist()
        if sum(trueMeal_List) > 0 and len(meal_List) >= 30:
            trueTime_Matrix.append(trueTime_List)
            trueMeal_Matrix.append(trueMeal_List)
            time_Matrix.append(time_List)
            meal_Matrix.append(meal_List)
    return meal_Matrix, time_Matrix, trueMeal_Matrix, trueTime_Matrix

meal_Matrix, time_Matrix, trueMeal_Matrix, trueTime_Matrix = extractingMealMatrix(timeCMG_Valid, timeCMG_Valid_Extra30, timeCMG_ValidStart)



# Bin Values based on time

In [9]:
carbMatrix = []
for j in trueTime_Matrix:
    dfI_carbTemp = dfI_carbinput.loc[dfI_carbinput['DateTime'] <= j[-1]]
    dfI_carbTemp = dfI_carbTemp.iloc[0]
    carbInputval = dfI_carbTemp['Carb Input']
    carbMatrix.append(carbInputval)


In [10]:
def carbBinMatrix(carbMin, carbMax, carbMatrix):
    binMatrix = []
    for i in carbMatrix:
        if i >= carbMin and i <= carbMin + 20:
            binMatrix.append(0)
        if i >= carbMin + 21 and i <= carbMin + 40:
            binMatrix.append(1)
        if i >= carbMin + 41 and i <= carbMin + 60:
            binMatrix.append(2)
        if i >= carbMin + 61 and i <= carbMin + 80:
            binMatrix.append(3)
        if i >= carbMin + 81 and i <= carbMin + 100:
            binMatrix.append(4)
        if i >= carbMin + 100 and i <= carbMin + 120:
            binMatrix.append(5)
    return binMatrix

binMatrix = carbBinMatrix(carbMin, carbMax, carbMatrix)
print(binMatrix)

carbdf = pd.DataFrame(binMatrix)
carbdf

[1, 0, 3, 0, 1, 4, 0, 1, 2, 0, 0, 1, 2, 0, 3, 2, 2, 0, 2, 0, 1, 2, 4, 0, 2, 0, 1, 0, 4, 0, 0, 4, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 1, 1, 4, 1, 0, 1, 2, 1, 0, 1, 3, 1, 0, 2, 0, 0, 1, 2, 2, 2, 1, 2, 0, 0, 0, 2, 1, 0, 3, 0, 0, 1, 1, 2, 0, 1, 2, 0, 1, 2, 0, 2, 2, 0, 1, 0, 0, 4, 1, 1, 1, 1, 0, 0, 3, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 4, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 1, 3, 2, 0, 0, 2, 0, 3, 1, 1, 1, 0, 0, 4, 0, 0, 0, 1, 3, 1, 2, 0, 3, 0, 0, 1, 0, 0, 2, 0, 1, 2, 1, 1, 2, 3, 0, 2, 2, 0, 1, 2, 1, 0, 1, 0, 1, 0, 1, 3, 1, 2, 3, 3, 1, 1, 4, 3, 0, 0, 0, 0, 1, 3, 0, 0, 2, 0, 1, 2, 0, 3, 1, 2, 1, 1, 0, 3, 2, 0, 0, 2, 0, 1, 2, 2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 4, 2, 0, 2, 2, 2, 0, 0, 2, 3, 0, 0, 0, 1, 1, 2, 0, 0, 3, 2, 1, 2, 1, 2, 0, 4, 0, 0, 1, 0, 1, 2, 1, 3, 0, 1, 0, 1, 2, 3, 3, 2, 1, 3, 0, 0, 3, 0, 0, 4, 1, 1, 0, 1, 2, 3, 0, 2, 3, 0, 3, 3, 1, 3, 2, 3, 3, 0, 1, 2, 1, 0, 4, 0, 1, 2, 2, 2, 1, 1, 1, 0, 1, 2, 0, 2, 1, 0, 1, 1, 1, 1, 2, 0, 2, 2, 4, 1, 2, 3, 4, 1, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 0, 

,0
0,1
1,0
2,3
3,0
4,1
...,...
465,2
466,1
467,2
468,2


# Feature Extraction 0
Extarcting Power Spectral Density

In [11]:
#Generating PSD for each value of P
from scipy import signal

def extarctPSD(meal_Matrix):
    psd1 = []
    psd2 = []
    psd3 = []
    for i in meal_Matrix:
        f, pxx = signal.periodogram(i)
        psd1.append(round(np.mean(pxx[0:5]), 3))
        psd2.append(round(np.mean(pxx[5:10]), 3))
        psd3.append(round(np.mean(pxx[10:16]), 3))
    return psd1, psd2, psd3

psd1, psd2, psd3 = extarctPSD(meal_Matrix)



# Feature Extraction I

Extracting Interquartile Range


In [12]:
#Extracting iqr for all values of P
from scipy.stats import iqr

def extractIQR(meal_Matrix):
    iqr_List = []
    for i in range(len(meal_Matrix)):
        val = iqr(meal_Matrix[i])
        iqr_List.append(val)
    return iqr_List

iqr_List = extractIQR(meal_Matrix)


# Feature Extraction II

Entropy for all P values


In [13]:
#Extracting Entropy for all P values
from scipy.stats import entropy

def extractEntropy(meal_Matrix):
    entropy_List = []
    for i in range(len(meal_Matrix)):
        val, count = np.unique(meal_Matrix[i], return_counts = True)
        entropy_List.append(round(entropy(count, None), 3))
    return entropy_List
        
entropy_List = extractEntropy(meal_Matrix)


# Feature Extraction III
Extracting peaks and frequencies unsinf FFT

In [14]:
#Extracting FFT values
def extractFFTValues(meal_Matrix):
    freq2 = []
    freq3 = []
    freq1 = []
    freq4 = []
    freq5 = []
    freq6 = []
    for i in range(len(meal_Matrix)):
        CGM_temp = meal_Matrix[i]
        fft = sp.fftpack.fft(CGM_temp)
        amp = np.abs(fft)
        psd = amp ** 2
        freq = sp.fftpack.fftfreq(len(CGM_temp), 1)
    
        #Sorting and finding max values
        amp_sort = sorted(amp)
        amp_sorted = []
        [amp_sorted.append(x) for x in amp_sort if x not in amp_sorted]
        
        pos_1, pos_2, pos_3 = amp_sorted[-2], amp_sorted[-3], amp_sorted[-4]
        pos_4, pos_5, pos_6 = amp_sorted[-5], amp_sorted[-6], amp_sorted[-7]

        #checking for peak index by position
        peak_1, peak_2 = np.where(amp == pos_1), np.where(amp == pos_2)
        peak_3, peak_4 = np.where(amp == pos_3), np.where(amp == pos_4)
        peak_5, peak_6 = np.where(amp == pos_5), np.where(amp == pos_6)
        
        
        #getting peak indices
        peak_index1, peak_index2 = peak_1[0][0], peak_2[0][0]
        peak_index3, peak_index4 = peak_3[0][0], peak_4[0][0]
        peak_index5, peak_index6 = peak_5[0][0], peak_6[0][0]
        
        #peak freq values
        freq_1, freq_2 = freq[peak_index1], freq[peak_index2]
        freq_3, freq_4 = freq[peak_index3], freq[peak_index4]
        freq_5, freq_6 = freq[peak_index5], freq[peak_index6]
        
        
        #adding into list
        freq2.append(round(freq_2, 3))
        freq3.append(round(freq_3, 3))
        freq1.append(round(freq_1, 3))
        freq4.append(round(freq_4, 3))
        freq5.append(round(freq_5, 3))
        freq6.append(round(freq_6, 3))
    return freq1, freq2, freq3, freq4, freq5, freq6

freq1, freq2, freq3, freq4, freq5, freq6 = extractFFTValues(meal_Matrix)



# Feature Extraction IV

Extarcting derivatives and double derivatives.

In [15]:
#Extracting first derivative
def extractFirstDervative(meal_Matrix):
    firstDervative_min = []
    firstDervative_max = []
    firstDervative_mean = []
    for i in range(len(meal_Matrix)):
        CGMmax = max(meal_Matrix[i])
        CGMmax_Index = meal_Matrix[i].index(CGMmax)
        diff = np.diff(meal_Matrix[i][:CGMmax_Index], 1)
        diff = diff.tolist()
        if len(diff) > 0:
            diff_avg = np.mean(diff)
            diff_max = max(diff)
            diff_min = min(diff)
        else:
            diff_avg = 0
            diff_max = 0
            diff_min = 0
        firstDervative_mean.append(round(diff_avg, 2))
        firstDervative_max.append(diff_max)
        firstDervative_min.append(diff_min)
    return firstDervative_mean, firstDervative_max, firstDervative_min

#Extracting second derivative
def extractSecondDervative(meal_Matrix):
    secondDervative_mean = []
    secondDervative_max = []
    secondDervative_min = []
    for i in range(len(meal_Matrix)):
        CGMmax = max(meal_Matrix[i])
        CGMmax_Index = meal_Matrix[i].index(CGMmax)
        diff = np.diff(meal_Matrix[i][0:CGMmax_Index], 2)
        if len(diff) > 0:
            diff_avg = np.mean(diff)
            diff_max = max(diff)
            diff_min = min(diff)
        else:
            diff_avg = 0
            diff_max = 0
            diff_min = 0
        secondDervative_mean.append(round(diff_avg, 2))
        secondDervative_max.append(diff_max)
        secondDervative_min.append(diff_min)
    return secondDervative_mean, secondDervative_max, secondDervative_min

firstDervative_mean, firstDervative_max, firstDervative_min  = extractFirstDervative(meal_Matrix)
secondDervative_mean, secondDervative_max, secondDervative_min = extractSecondDervative(meal_Matrix)



# Creating Meal Feature Matrix

In [16]:
#Creating Feature Matrices:

data =[]
for i in range(len(meal_Matrix)):
    data.append({
    'Interquartile Range' : iqr_List[i],
    'Entropy' : entropy_List[i],
    'PSD I' : psd1[i],
    'PSD II' : psd2[i],
    'PSD III' : psd3[i],
    'Frequency I' : freq1[i],
    'Frequency II' : freq2[i],
    'Frequency III' : freq3[i],
    'Frequency IV' : freq4[i],
    'Frequency V' : freq5[i],
    'Frequency VI' : freq6[i],
    'Velocity Mean' : firstDervative_mean[i],
    'Velocity Max' : firstDervative_max[i],
    'Velocity Min' : firstDervative_min[i],
    'Accelaration Mean' : secondDervative_mean[i],
    'Accelaration Max' : secondDervative_max[i],
    'Accelaration Min' : secondDervative_min[i]
        })

mealData_Feature_Matrix = pd.DataFrame(data)
meal_feature_matrix = mealData_Feature_Matrix.reset_index().drop(columns = 'index')



# Normalizing the data frame

In [17]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
meal_feature_matrix.iloc[:,:] = scaler.fit_transform(meal_feature_matrix.iloc[:,:].to_numpy())


# K-Means Clustering

In [18]:
#First find the optimal number of clusters by using elbow method
eachCluster = []
for i in range(1,15):
    kmeans = KMeans(n_clusters=i, init = 'k-means++', max_iter = 300, n_init= 10)
    kmeans.fit(meal_feature_matrix[0::].values)
    eachCluster.append(kmeans.inertia_)

#plt.plot(range(1,15), eachCluster)
#plt.title("Cluster Efficiency")
#plt.xlabel("Number of Clusters")
#plt.ylabel("Each Cluster")
#plt.show()


Based on the graph above it is evident that having 6 clusters will give optimal results.

In [19]:
# Creating 6 clusters for the dataset
kmeans = KMeans(n_clusters = 6, max_iter=300, n_init= 10)
cluster_KMeans = kmeans.fit_predict(meal_feature_matrix)
labels = kmeans.predict(meal_feature_matrix)

kmeans_matrix = meal_feature_matrix.copy()
kmeans_matrix['Cluster'] = pd.Series(labels, index = meal_feature_matrix.index)



# KMeans Cluster Analysis

In [20]:
#Calculating SSE
cluster_centers = [kmeans_matrix[kmeans.labels_ == i].mean(axis=0) for i in range(6)]

sseKMeans = [0,0,0,0,0,0]
for point, label in zip(kmeans_matrix.values, kmeans.labels_):
    sseKMeans[label] += np.square(point - cluster_centers[label]).sum()
    
KMeans_SSE = round(sum(sseKMeans), 3)
    
#First creating confusion matrix for calculating Purity and Entropy
tempdf = pd.DataFrame()
tempdf['Ground Truth'] = carbdf
tempdf['Clusters'] = list(labels)

cf_matrix = pd.pivot_table(tempdf, index='Clusters', columns='Ground Truth', aggfunc=len)

cf_matrix.fillna(value = 0, inplace = True)

#temp df to store total values
cf_matrix_temp = cf_matrix.copy()

cf_matrix_temp['Total'] = cf_matrix.sum(axis=1)
total = cf_matrix_temp['Total']

print(cf_matrix)

Ground Truth     0     1     2     3     4    5
Clusters                                       
0             32.0  24.0  15.0   7.0   6.0  1.0
1              8.0   8.0  17.0   8.0   4.0  1.0
2             53.0  39.0  39.0  22.0  10.0  1.0
3              8.0   7.0   5.0   3.0   3.0  0.0
4              3.0   8.0   9.0   2.0   1.0  0.0
5             44.0  43.0  25.0  10.0   3.0  1.0


In [21]:
#Calculating Entropy and Purity
entropyKmeans = []
purityKmeans = []
for row, rowData in cf_matrix_temp.iterrows():
    entropy = 0
    purity = []
    for i in rowData[0:5]:
        if i > 0:
            val = round(i/rowData.iloc[6] , 3)
            purity.append(val)
            entropy += val + math.log2(val)
    entropyKmeans.append(-entropy)
    purityKmeans.append(max(purity))
    
#calculating total entropy, purity and sse
KMeans_Entropy = 0
KMeans_Purity = 0
val1 = 0
val2 = 0
for i in range(len(entropyKmeans)):
    val1 += total.values[i] * entropyKmeans[i]
    val2 += total.values[i] * purityKmeans[i]
KMeans_Entropy = round((val1/len(meal_feature_matrix)), 3)
KMeans_Purity = round((val2/len(meal_feature_matrix)), 3)


# DBScan Clustering

In [22]:
#Execute dbscan for meal_feature_matrix
dbscan = DBSCAN(eps=0.211, min_samples=5).fit(meal_feature_matrix)
cluster_DBScan = dbscan.fit_predict(meal_feature_matrix)

cluster_labels = np.unique(dbscan.labels_)
cluster_labels = labels[1:]


# DBScan Cluster Analysis

In [23]:
#Calculating SSE
dbscan_matrix = meal_feature_matrix.copy()
dbscan_matrix['db-cluster'] = dbscan.labels_

cluster_num = len(set(dbscan.labels_)) -(1 if -1 in dbscan.labels_ else 0)
cluster_centers_dbs = [dbscan_matrix[dbscan.labels_ == i].mean(axis=0) for i in range(cluster_num +1)]

sseDBScan = [0,0,0,0,0,0,0,0]
for point, label in zip(dbscan_matrix.values, dbscan.labels_):
    sseDBScan[label] += np.square(point - cluster_centers[label]).sum()

sseDBScan = sseDBScan[0:6]

DBScan_SSE = round(sum(sseDBScan), 3)


In [24]:
#Creating Confusion Matrix
tempdf = pd.DataFrame()
tempdf['Ground Truth'] = carbdf
tempdf['Clusters'] = list(dbscan.labels_)

dbs_matrix = pd.pivot_table(tempdf, index='Clusters', columns='Ground Truth', aggfunc=len)

dbs_matrix.fillna(value = 0, inplace = True)
dbs_matrix = dbs_matrix.drop(dbs_matrix.index[0])

#temp df to store total values
dbs_matrix_temp = dbs_matrix.copy()
dbs_matrix_temp['Total'] = dbs_matrix_temp.sum(axis=1)

total1 = dbs_matrix_temp['Total']

print(dbs_matrix)

Ground Truth     0     1     2     3    4    5
Clusters                                      
0             35.0  32.0  22.0  12.0  6.0  1.0
1              3.0   1.0   0.0   0.0  1.0  0.0
2              4.0   4.0   3.0   1.0  0.0  0.0
3              2.0   1.0   2.0   0.0  0.0  0.0
4              2.0   1.0   0.0   1.0  1.0  0.0
5              2.0   3.0   0.0   0.0  0.0  0.0


In [25]:
#Calculating Entropy and Purity
entropyDBScan = []
purityDBScan = []
for row, rowData in dbs_matrix_temp.iterrows():
    entropy = 0
    purity = []
    for i in rowData[0:5]:
        if i > 0:
            val = round(i/rowData.iloc[6] , 3)
            purity.append(val)
            entropy += val + math.log2(val)
    entropyDBScan.append(-entropy)
    purityDBScan.append(max(purity))

#calculating total entropy, purity and sse
DBScan_Entropy = 0
DBScan_Purity = 0
val1 = 0
val2 = 0
for i in range(len(purityDBScan)):
    val1 += total1.values[i] * entropyDBScan[i]
    val2 += total1.values[i] * purityDBScan[i]
DBScan_Entropy = round((val1/len(meal_feature_matrix)), 3)
DBScan_Purity = round((val2/len(meal_feature_matrix)), 3)


# Creating Results.csv

In [26]:
results = [KMeans_SSE, DBScan_SSE, KMeans_Entropy, DBScan_Entropy, KMeans_Purity, DBScan_Purity]
resultsdf = pd.DataFrame(results).T

resultsdf.to_csv('Results.csv', header = False, index = False)